In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Plotly settings
import plotly.io as pio
pio.templates.default = "plotly_white"

# 1. EDA

In [2]:
df = pd.read_csv('../data/raw/household_power_consumption.csv', 
                 sep=',', 
                 na_values='?',          
                 dayfirst=True,
                 low_memory=False)

In [3]:
df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [4]:
df.describe()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
count,1.044506e+06,1.044506e+06,1.044506e+06,1.044506e+06,1.044506e+06,1.044506e+06,1.044506e+06
mean,1.108202e+00,1.182732e-01,2.399598e+02,4.718123e+00,1.177451e+00,1.475177e+00,5.933837e+00
std,1.129546e+00,1.095062e-01,3.284504e+00,4.763588e+00,6.321183e+00,6.352703e+00,8.210398e+00
min,7.600000e-02,0.000000e+00,2.234900e+02,2.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.880000e-01,0.000000e+00,2.379700e+02,1.200000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.500000e-01,9.800000e-02,2.402100e+02,2.400000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.544000e+00,1.860000e-01,2.421400e+02,6.400000e+00,0.000000e+00,1.000000e+00,1.700000e+01
max,1.067000e+01,1.390000e+00,2.521400e+02,4.640000e+01,8.000000e+01,7.800000e+01,3.100000e+01


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 9 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Date                   1048575 non-null  object 
 1   Time                   1048575 non-null  object 
 2   Global_active_power    1044506 non-null  float64
 3   Global_reactive_power  1044506 non-null  float64
 4   Voltage                1044506 non-null  float64
 5   Global_intensity       1044506 non-null  float64
 6   Sub_metering_1         1044506 non-null  float64
 7   Sub_metering_2         1044506 non-null  float64
 8   Sub_metering_3         1044506 non-null  float64
dtypes: float64(7), object(2)
memory usage: 72.0+ MB


# 2. Data cleaning and preprocessing

In [6]:
df.isnull().sum()

Date                        0
Time                        0
Global_active_power      4069
Global_reactive_power    4069
Voltage                  4069
Global_intensity         4069
Sub_metering_1           4069
Sub_metering_2           4069
Sub_metering_3           4069
dtype: int64

In [7]:
df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], 
                                  format='%d/%m/%Y %H:%M:%S')

df = df.set_index('datetime')
df = df.drop(['Date', 'Time'], axis=1)

df = df.replace('?', np.nan)
df = df.apply(pd.to_numeric, errors='coerce')

In [8]:
df_clean = df.copy()

df_clean = df_clean.fillna(method='ffill', limit=60)
df_clean = df_clean.interpolate(method='time')
df_clean = df_clean.dropna()

In [9]:
# Business metrics
avg_power_kw = df_clean['Global_active_power'].mean()
avg_power_daily_kwh = avg_power_kw * 24  # kW * 24h = kWh per day
avg_power_yearly_kwh = avg_power_daily_kwh * 365

# Austrian electricity cost: ~0.25 €/kWh (2024)
ELECTRICITY_COST = 0.25  # EUR per kWh
yearly_cost = avg_power_yearly_kwh * ELECTRICITY_COST

In [10]:
df_hourly = df_clean.resample('H').mean()

# Daily aggregation
df_daily = df_clean.resample('D').agg({
    'Global_active_power': ['mean', 'sum', 'std'],
    'Sub_metering_1': 'sum',
    'Sub_metering_2': 'sum',
    'Sub_metering_3': 'sum'
})

# 3. Visualization

## VISUALIZATION 1: OVERVIEW TIME SERIES

In [11]:
sample_week = df_hourly['2007-01-01':'2007-01-07']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=sample_week.index,
    y=sample_week['Global_active_power'],
    mode='lines',
    name='Global Active Power',
    line=dict(color='#1f77b4', width=2)
))

fig.update_layout(
    title='Hourly Power Consumption - Sample Week (Jan 2007)',
    xaxis_title='Date',
    yaxis_title='Power (kW)',
    height=500,
    hovermode='x unified'
)

fig.show()

## VISUALIZATION 2: DAILY PATTERNS

In [12]:
df_hourly['hour'] = df_hourly.index.hour
hourly_pattern = df_hourly.groupby('hour')['Global_active_power'].mean()

fig = go.Figure()

fig.add_trace(go.Bar(
    x=hourly_pattern.index,
    y=hourly_pattern.values,
    marker_color='#2ca02c',
    text=hourly_pattern.values.round(2),
    textposition='outside'
))

fig.update_layout(
    title='Average Power Consumption by Hour of Day',
    xaxis_title='Hour',
    yaxis_title='Average Power (kW)',
    height=500,
    xaxis=dict(tickmode='linear', tick0=0, dtick=1)
)

fig.show()

# Business insight
peak_hour = hourly_pattern.idxmax()
peak_power = hourly_pattern.max()
offpeak_hour = hourly_pattern.idxmin()
offpeak_power = hourly_pattern.min()

print(f"\nPEAK LOAD ANALYSIS:")
print(f"Peak hour: {peak_hour}:00 ({peak_power:.2f} kW)")
print(f"Off-peak hour: {offpeak_hour}:00 ({offpeak_power:.2f} kW)")
print(f"Peak vs. Off-peak difference: {((peak_power/offpeak_power - 1) * 100):.1f}%")


PEAK LOAD ANALYSIS:
Peak hour: 21:00 (1.99 kW)
Off-peak hour: 4:00 (0.44 kW)
Peak vs. Off-peak difference: 356.2%


## VISUALIZATION 3: WEEKLY PATTERNS

In [13]:
df_hourly['day_of_week'] = df_hourly.index.dayofweek
df_hourly['day_name'] = df_hourly.index.day_name()

weekly_pattern = df_hourly.groupby('day_name')['Global_active_power'].mean()
# Reorder to start with Monday
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekly_pattern = weekly_pattern.reindex(day_order)

fig = go.Figure()

fig.add_trace(go.Bar(
    x=weekly_pattern.index,
    y=weekly_pattern.values,
    marker_color=['#d62728' if day in ['Saturday', 'Sunday'] else '#1f77b4' 
                  for day in weekly_pattern.index],
    text=weekly_pattern.values.round(2),
    textposition='outside'
))

fig.update_layout(
    title='Average Power Consumption by Day of Week',
    xaxis_title='Day',
    yaxis_title='Average Power (kW)',
    height=500
)

fig.show()

print(f"\n WEEKLY PATTERN:")
print(f"Weekday avg: {weekly_pattern[:'Friday'].mean():.2f} kW")
print(f"Weekend avg: {weekly_pattern['Saturday':].mean():.2f} kW")


 WEEKLY PATTERN:
Weekday avg: 1.04 kW
Weekend avg: 1.27 kW


## VISUALIZATION 4: SUB-METERING BREAKDOWN

In [14]:
sub_metering_totals = {
    'Kitchen': df_clean['Sub_metering_1'].sum() / 1000,  # Convert to kWh
    'Laundry': df_clean['Sub_metering_2'].sum() / 1000,
    'Climate Control': df_clean['Sub_metering_3'].sum() / 1000
}

# Calculate other (unmetered)
total_consumption = df_clean['Global_active_power'].sum() * (1/60)  # Convert minutes to hours
metered_total = sum(sub_metering_totals.values())
sub_metering_totals['Other Appliances'] = total_consumption - metered_total

fig = go.Figure(data=[go.Pie(
    labels=list(sub_metering_totals.keys()),
    values=list(sub_metering_totals.values()),
    hole=0.4,
    marker_colors=['#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
)])

fig.update_layout(
    title='🏠 Energy Consumption Breakdown by Area',
    height=500,
    annotations=[dict(text='Total<br>Consumption', x=0.5, y=0.5, font_size=16, showarrow=False)]
)

fig.show()

# Business insight
for area, kwh in sub_metering_totals.items():
    cost = kwh * 0.25
    print(f"{area}: {kwh:,.0f} kWh → €{cost:,.2f}")

Kitchen: 1,230 kWh → €307.49
Laundry: 1,541 kWh → €385.26
Climate Control: 6,200 kWh → €1,550.09
Other Appliances: 10,351 kWh → €2,587.70


## VISUALIZATION 5: SEASONAL PATTERNS

In [15]:
df_daily['month'] = df_daily.index.month
df_daily['year'] = df_daily.index.year

monthly_pattern = df_clean.resample('M').agg({
    'Global_active_power': 'mean'
})

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=monthly_pattern.index,
    y=monthly_pattern['Global_active_power'],
    mode='lines+markers',
    line=dict(color='#17becf', width=2),
    marker=dict(size=6)
))

fig.update_layout(
    title='📈 Monthly Average Power Consumption Trend',
    xaxis_title='Date',
    yaxis_title='Average Power (kW)',
    height=500,
    hovermode='x unified'
)

fig.show()

print(f"\n🔍 SEASONAL PATTERN:")
summer_months = [6, 7, 8]
winter_months = [12, 1, 2]

df_with_month = df_clean.copy()
df_with_month['month'] = df_with_month.index.month

summer_avg = df_with_month[df_with_month['month'].isin(summer_months)]['Global_active_power'].mean()
winter_avg = df_with_month[df_with_month['month'].isin(winter_months)]['Global_active_power'].mean()

print(f"Summer avg (Jun-Aug): {summer_avg:.2f} kW")
print(f"Winter avg (Dec-Feb): {winter_avg:.2f} kW")
print(f"Seasonal difference: {((winter_avg/summer_avg - 1) * 100):.1f}%")


🔍 SEASONAL PATTERN:
Summer avg (Jun-Aug): 0.72 kW
Winter avg (Dec-Feb): 1.48 kW
Seasonal difference: 105.8%


## CORRELATION ANALYSIS

In [16]:
features_for_corr = ['Global_active_power', 'Global_reactive_power', 
                     'Voltage', 'Global_intensity',
                     'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']

corr_matrix = df_clean[features_for_corr].corr()

fig = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.columns,
    colorscale='RdBu',
    zmid=0,
    text=corr_matrix.values.round(2),
    texttemplate='%{text}',
    textfont={"size": 10}
))

fig.update_layout(
    title='Feature Correlation Matrix',
    height=600,
    width=700
)

fig.show()

## KEY INSIGHTS SUMMARY

In [17]:
print("\n" + "=" * 70)
print("🎯 KEY INSIGHTS FOR BUSINESS CASE")
print("=" * 70)

print(f"""
1. CONSUMPTION PATTERNS:
   • Peak hours: 18:00-21:00 (evening)
   • Off-peak: 02:00-05:00 (night)
   • Potential for load-shifting: {((peak_power/offpeak_power - 1) * 100):.0f}%
   
2. SEASONAL VARIATION:
   • Winter consumption {((winter_avg/summer_avg - 1) * 100):.0f}% higher than summer
   • Heating/cooling drives costs
   
3. ENERGY BREAKDOWN:
   • Kitchen: {sub_metering_totals['Kitchen']/total_consumption*100:.1f}%
   • Laundry: {sub_metering_totals['Laundry']/total_consumption*100:.1f}%
   • Climate Control: {sub_metering_totals['Climate Control']/total_consumption*100:.1f}%
   
4. BUSINESS OPPORTUNITY:
   • Current yearly cost: €{yearly_cost:.2f}
   • With 15% optimization: €{yearly_cost * 0.85:.2f}
   • Potential savings: €{yearly_cost * 0.15:.2f}/year
   • ROI: < 12 months (assuming smart meter cost €100-200)
""")

print("=" * 70)


🎯 KEY INSIGHTS FOR BUSINESS CASE

1. CONSUMPTION PATTERNS:
   • Peak hours: 18:00-21:00 (evening)
   • Off-peak: 02:00-05:00 (night)
   • Potential for load-shifting: 356%
   
2. SEASONAL VARIATION:
   • Winter consumption 106% higher than summer
   • Heating/cooling drives costs
   
3. ENERGY BREAKDOWN:
   • Kitchen: 6.4%
   • Laundry: 8.0%
   • Climate Control: 32.1%
   
4. BUSINESS OPPORTUNITY:
   • Current yearly cost: €2421.32
   • With 15% optimization: €2058.12
   • Potential savings: €363.20/year
   • ROI: < 12 months (assuming smart meter cost €100-200)



## SAVE KEY METRICS FOR LATER USE

In [18]:
metrics = {
    'yearly_cost': yearly_cost,
    'peak_hour': int(peak_hour),
    'peak_power': peak_power,
    'offpeak_hour': int(offpeak_hour),
    'offpeak_power': offpeak_power,
    'summer_avg': summer_avg,
    'winter_avg': winter_avg,
    'sub_metering': sub_metering_totals
}

import joblib
joblib.dump(metrics, '../data/processed/baseline_metrics.pkl')

['../data/processed/baseline_metrics.pkl']